In [1]:
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import math

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
# original group

# group = [
#     [4],[10, 11, 12, 41],[34],[40, 42]
#     ,[6, 8, 13, 17, 25, 26, 48, 53, 55, 56],[7, 18],[27, 57],[35, 46, 47]
#     ,[1, 31],[9, 32],[3],[33]
#     ,[2, 14, 22, 23, 37, 44, 52, 54],[15],[38, 58, 43],[29, 39],[45]
#     ,[5, 59],[16, 24],[19, 20, 21, 49, 50, 51],[28, 30, 36, 60]
# ]

# group_name = [
#     "A1", "A2", "A3", "A4", 
#     "B1", "B2", "B3", "B4",
#     "C1", "C2", "C3", "C4", 
#     "D1", "D2", "D3", "D4", "D5", 
#     "E1", "E2", "E3", "E4"
# ]

# save_path = f"Refined_Data/Grouped_Data/Input_Data.csv"

# group 2

group = [
    [4],[11, 12],[34],[40],[10],[42],[41]
    ,[6, 8, 13, 17, 26, 48, 53, 55, 56],[7, 18],[27, 57],[35, 46, 47],[25]
    ,[1, 31],[9, 32],[3],[33]
    ,[2, 14, 22, 37, 44, 52, 54],[15],[38, 58, 43],[29, 39],[45],[23]
    ,[5],[16, 24],[19, 20, 21, 49, 50, 51],[28, 30, 36, 60],[59]
]

group_name = [
    "A1", "A2", "A3", "A4", "A5", "A6", "A7",
    "B1", "B2", "B3", "B4", "B5",
    "C1", "C2", "C3", "C4", 
    "D1", "D2", "D3", "D4", "D5", "D6", 
    "E1", "E2", "E3", "E4", "E5"
]

save_path = f"Refined_Data/Grouped_Data/Input_Data2.csv"

In [3]:
def get_pow(series):
    return math.pow(series, 0.16)

In [4]:
data_path = "./data/train.csv"

data = pd.read_csv(data_path, parse_dates = ["date_time"])

data["time_idx"] = 0
data["month"] = 0
data["energy_group"] = 0
data["Week"] = 0
data["24Hour"] = 0
data["holiday"] = 0
data["Weekend"] = 0

for i in range(len(group)):
    data.loc[data["num"].isin(group[i]), "energy_group"] = str(i)

data.loc[data["date_time"].dt.month == 6, 'month'] = 0
data.loc[data["date_time"].dt.month == 7, 'month'] = 30
data.loc[data["date_time"].dt.month == 8, 'month'] = 61

data.loc[(data["date_time"].dt.month == 8) & (data["date_time"].dt.day == 17) , 'holiday'] = 1

data["time_idx"] = data["date_time"].dt.hour + data["date_time"].dt.day * (24) + data["month"] * 24
data["time_idx"] = data["time_idx"] - min(data["time_idx"])

data["Week"] = (data["date_time"].dt.day + data["month"]) % 7
data["24Hour"] = data["date_time"].dt.hour

data.loc[data["Week"] == 6, 'Weekend'] = 1
data.loc[data["Week"] == 0, 'Weekend'] = 1

data = data.drop("month",axis='columns')

data.rename(columns = {'전력사용량(kWh)' : 'kWH', '기온(°C)' : 'C', '풍속(m/s)' : 'm/s', '습도(%)' : 'wet', '강수량(mm)' : 'mm','일조(hr)' : 'hr', '비전기냉방설비운영' : "non_electric_aircondition", "태양광보유" : "sunlight"}, inplace = True)

# 체감 온도, 불쾌 지수
data['perceived_temperature'] = 13.12 + 0.6215*data['C'] - 11.37*data['m/s'].apply(get_pow) + 0.3965*data['m/s'].apply(get_pow)*data['C']
data['discomfort_index'] = (1.8 * data['C']) - 0.55 * (1 - (data['wet'] * 0.01)) * (1.8 * data['C'] - 26) + 32

data = data.astype({'non_electric_aircondition' : int, 'sunlight' : int})
data['num'] = data['num'].apply(str)
data['Week'] = data['Week'].apply(str)
data['24Hour'] = data['24Hour'].apply(str)
data['holiday'] = data['holiday'].apply(str)
data['Weekend'] = data['Weekend'].apply(str)

data.to_csv(save_path, mode='w')